In [32]:
import geopandas as gpd
import pandas as pd
from blocksnet import City

blocks = gpd.read_parquet('blocks.parquet')
adj_mx = pd.read_pickle('adj_mx.pickle')

city = City(blocks, adj_mx)

## Buildings

In [33]:
buildings = gpd.read_parquet('data/platform/buildings.parquet')
buildings = buildings.rename(columns={
  'storeys_count': 'number_of_floors',
  'population_balanced': 'population',
}).to_crs(city.crs)
buildings['number_of_floors'] = buildings['number_of_floors'].fillna(1)
buildings['build_floor_area'] = buildings.apply(lambda s : s.geometry.area * s.number_of_floors, axis=1)
buildings['living_area'] = buildings.apply(lambda s : s.build_floor_area if s.is_living else 0, axis=1)
buildings['non_living_area'] = buildings['build_floor_area'] - buildings['living_area']
buildings.head()

,geometry,address,building_id,building_area,living_area,number_of_floors,population,central_heating,central_water,central_hot_water,...,properties,building_year,refusechute,resident_number,ukname,lift_count,project_type,repair_years,build_floor_area,non_living_area
0,"POLYGON ((360058.761 6654864.061, 360055.136 6...",None,1921264,95.155945,95.125499,1.0,3,False,False,False,...,{'new': None},NaN,None,NaN,None,NaN,None,None,95.125499,0.0
1,"POLYGON ((359989.536 6654834.089, 359998.291 6...",None,1921265,97.709000,97.677776,1.0,0,False,False,False,...,{'new': None},NaN,None,NaN,None,NaN,None,None,97.677776,0.0
2,"POLYGON ((358920.466 6654189.968, 358910.608 6...",None,1921319,706.651001,14128.607389,20.0,144,False,False,False,...,{'new': None},NaN,None,NaN,None,NaN,None,None,14128.607389,0.0
3,"POLYGON ((358923.144 6653917.733, 358921.359 6...",None,1921320,721.190125,18024.123801,25.0,235,False,False,False,...,{'new': None},NaN,None,NaN,None,NaN,None,None,18024.123801,0.0
4,"POLYGON ((358956.348 6653979.711, 358969.230 6...",None,1921321,1543.230220,32397.704978,21.0,337,False,False,False,...,{'new': None},NaN,None,NaN,None,NaN,None,None,32397.704978,0.0


In [34]:
city.update_buildings(buildings)

Update blocks buildings: 100%|██████████| 2971/2971 [00:15<00:00, 187.61it/s]


## Services

In [43]:
school = gpd.read_parquet('data/platform/school.parquet')
school['area'] = school.area
school.geometry = school.geometry.representative_point()

In [44]:
city.update_services('school', school.to_crs(city.crs))

Update buildings services: 100%|██████████| 644/644 [00:00<00:00, 811.93it/s]
Update blocks services: 0it [00:00, ?it/s]/home/vasilstar/masterplanning/venv/lib/python3.10/site-packages/tqdm/std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
Update blocks services: 0it [00:00, ?it/s]


In [45]:
city.get_services_gdf()

,geometry,block_id,building_id,service_type,capacity,area,is_integrated
0,POINT (355775.115 6648216.661),0,111219,school,960,1773.412612,False
1,POINT (358215.654 6638851.805),18,52264,school,998,1909.694800,False
2,POINT (350253.465 6640672.786),25,142462,school,140,1324.534116,False
3,POINT (350562.659 6640676.973),26,46689,school,163,1697.319143,False
4,POINT (351481.263 6647741.719),30,124477,school,848,1559.776279,False
...,...,...,...,...,...,...,...
642,POINT (352978.841 6647290.173),3724,128242,school,164,2348.555760,False
643,POINT (353488.497 6647200.202),3730,155516,school,970,926.035859,False
644,POINT (348150.051 6660928.956),3746,157506,school,619,5026.649853,False
645,POINT (337995.656 6637224.912),3751,25285,school,322,8182.061229,False
